In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [22]:
cd content/drive/MyDrive/Project_DIP/Dataset

/content/drive/MyDrive/Project_DIP/Dataset


In [25]:
directories = os.listdir()
filename = []
label = []

for i in range(0, len(directories)):
  os.chdir(directories[i])
  files = os.listdir()
  for j in range(0, len(files)):
    files[j] = directories[i] + '/' + files[j]
    label.append(directories[i])
  filename = filename + files
  os.chdir("..")
print(filename)
print(label)

['scissors/0Ug54ifXRqqlZS2Z.png', 'scissors/0Flw60Z2MAWWKn6S.png', 'scissors/0ePX1wuCc3et7leL.png', 'scissors/0657zSfiTYzP2jrl.png', 'scissors/0CSaM2vL2cWX6Cay.png', 'scissors/3kVoftgTNdDcvPPY.png', 'scissors/3dI7E5pwR07mvKvF.png', 'scissors/3hIcgfPOoWEIs16j.png', 'scissors/3bkC6JAPog7xX9WO.png', 'scissors/2vDaPrc35RGC8nvM.png', 'scissors/2ZPcSpOm7SEfQYwg.png', 'scissors/2V5E7uHmAh9eK0qt.png', 'scissors/2ufDQYIqKG1xRG7y.png', 'scissors/2tRxoWcFfrvjsHTZ.png', 'scissors/2TAGoXw7yaK0bXBu.png', 'scissors/2M8LvUBGMOH1bsaz.png', 'scissors/2J7q0JbqJrjmZC05.png', 'scissors/2gd0aqAxW55bHZP9.png', 'scissors/2l1K148aIJHRR1q7.png', 'scissors/2fxAdPTgrVIoITsL.png', 'scissors/2E8j595xseKRF60J.png', 'scissors/2EeIeEe1P0a0Fi9v.png', 'scissors/2DEYFsJ27UgVqv3W.png', 'scissors/2C9FEbBklwcVhF3W.png', 'scissors/277q9TUTAsBAbTCj.png', 'scissors/256MrhkDete6aQhP.png', 'scissors/1yDUM1w2HTbuPIfO.png', 'scissors/1wZUIsowmZRmESjh.png', 'scissors/1xkTkvyzmavV7lQ5.png', 'scissors/1WX9KKoq0nkWiTxI.png', 'scissors

In [26]:
data = {"filename" : filename, 
        "label": label 
        }
df = pd.DataFrame(data=data)

df_shuffled = df.sample(frac=1)
dataset = df_shuffled
dataset

,filename,label
1731,paper/Gc4hUsoFAQBdulGg.png,paper
137,scissors/AORIZvib6x3RaA82.png,scissors
1138,rock/kcZfTQqKr8VaSjhk.png,rock
501,scissors/utufwM3u0nx14c52.png,scissors
1281,rock/QKlibCXvATAv2H4i.png,rock
...,...,...
436,scissors/XPvABb1iOSQx69Dq.png,scissors
531,scissors/tVigGbWgM7Rp9ic4.png,scissors
136,scissors/AOao5nWv96N9SV7O.png,scissors
1301,rock/SE5eUJl8euX8gTJ3.png,rock


##Yang bawah lom ditest

In [27]:
def load_images_files(train_data):
  images = []
  Y = []
  for obj in train_data.values:
    img = cv2.imread(obj[0])

    #interpolation, metode resize
    img_pred = cv2.resize(img, (64,64), interpolation = cv2.INTER_AREA)
    img_pred = img_pred / 255
    img_pred = img_pred.reshape(img_pred.shape[0]*img_pred.shape[1]*img_pred.shape[2])
    images.append(img_pred)
    Y.append(obj[1])
  X = np.array(images)
  Y = np.array(Y)
  return X, Y

In [28]:
x, y = load_images_files(dataset)
print(x.shape)
print(y.shape)

KeyboardInterrupt: ignored

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=21, test_size=0.4)

print(x.shape)
print(y.shape)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

for i in range(1, 16):
  knn = KNeighborsClassifier(n_neighbors=i, metric='euclidean')
  knn.fit(x_train, y_train)
  y_pred = knn.predict(x_test)
  print('K=',i,"Accuracy Score:",accuracy_score(y_test, y_pred))

In [ ]:
def load_single_images_files(path):
  images = []
  img = cv2.imread(path)
  img_pred = cv2.resize(img, (64,64), interpolation = cv2.INTER_AREA)
  img_pred = img_pred / 255
  img_pred = img_pred.reshape(img_pred.shape[0]*img_pred.shape[1]*img_pred.shape[2])
  images.append(img_pred)
  X = np.array(images)
  return X

In [ ]:
X_single = load_single_images_files('Cat/0.jpg')
jawaban_prediksi = knn.predict(X_single)
print(jawaban_prediksi)